In [2]:
# google drive mount

from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
!pip install gluonnlp==0.10.0 pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch==1.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 27.9 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595728 sha256=de8bd355ca6567aec613c7a6ad3ee9a16003a2f186350dd8c839a4ab6778c0af
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 27.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [4]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-vfkvza8j
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-vfkvza8j
     |████████████████████████████████| 129 kB 23.8 MB/s 
     |████████████████████████████████| 54.7 MB 1.3 MB/s 
     |████████████████████████████████| 4.5 MB 60.2 MB/s 
     |████████████████████████████████| 2.5 MB 80.3 MB/s 
     |████████████████████████████████| 6.7 MB 48.7 MB/s 
     |████████████████████████████████| 73 kB 2.2 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15708 sha256=e2a24f15e535578ec949f86c18ced517838e23b6373fafbc54e099f0e2b4c402
  Stored in directory: /tmp/pip-ephem-wheel-cache-uxcxd79r/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: transformers
    F

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

# GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
%cd '/gdrive/My Drive/KoBERT/checkpoint/'

/gdrive/My Drive/KoBERT/checkpoint


In [7]:
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

bertmodel, vocab = get_pytorch_kobert_model()

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /gdrive/My Drive/KoBERT/checkpoint/.cache/kobert_v1.zip
using cached model. /gdrive/My Drive/KoBERT/checkpoint/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /gdrive/My Drive/KoBERT/checkpoint/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [8]:
# data loading

import pandas as pd

# dataset = pd.read_csv('/gdrive/My Drive/KoBERT/data/data_3.csv') # 특수문자 제거 + 맞춤법 교정 + 조사,어미 제거
dataset = pd.read_csv('/gdrive/My Drive/KoBERT/data/data_2.csv') # 특수문자 제거 + 맞춤법 교정
dataset.head()

,sentence,emotion
0,제 감정이 이상해진 것 같아요 남편만 보면 화가 치밀어 오르고 감정 조절이 안돼요,화남
1,더 이상 내 감정을 내가 컨트롤 못 하겠어,비참함
2,하루 종일 오르락내리락 롤러코스터 타는 기분이에요,비참함
3,꼭 롤러코스터 타는 것 같아요,긴장됨
4,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요,우울함


In [9]:
# emotion labeling

emotion_labels = {'중립': 0,
                  '기쁨': 1,
                  '놀람': 2,
                  '긴장됨': 3,
                  '괴로움': 4,
                  '화남': 5,
                  '비참함': 6,
                  '우울함': 7,
                  '피로함': 8}

data_list = []
for q, label in zip(dataset['sentence'], dataset['emotion']):
  data = []
  data.append(q)
  data.append(str(emotion_labels[label]))

  data_list.append(data)

In [10]:
# tokenizing
from torch.utils.data import Dataset, DataLoader

class EmotionDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, 
                                               max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair) 

    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i], ))

  def __len__(self):
    return (len(self.labels))

In [11]:
# parameters

max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [12]:
# train/test 분리

from sklearn.model_selection import train_test_split

train, test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=42)
print("train:", len(train))
print("test:", len(test))

train: 38992
test: 9748


In [13]:
# train/test data tokenizing

data_train = EmotionDataset(train, 0, 1, tok, vocab, max_len, True, False)
data_test = EmotionDataset(test, 0, 1, tok, vocab, max_len, True, False)

In [14]:
# make torch type dataset

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [15]:
# classifier
import torch
from torch import nn

class EmotionClassifier(nn.Module):
  def __init__(self,
               bert,
               hidden_size = 768,
               num_classes = 9,
               dr_rate = None,
               params = None):
    super(EmotionClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size, num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids: torch.Tensor, valid_length: torch.Tensor, segment_ids: torch.Tensor):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    
    _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), 
                          attention_mask=attention_mask.float().to(token_ids.device), return_dict=False)
    
    if self.dr_rate:
      out = self.dropout(pooler)
    
    return self.classifier(out)

In [18]:
model = EmotionClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer and schedule
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# accuracy calculate
def calc_accuracy(X, y):
  max_vals, max_indices = torch.max(X, 1)
  train_acc = (max_indices == y).sum().data.cpu().numpy()/max_indices.size()[0]
  return train_acc

In [19]:
# train

train_list = []
test_list = []
loss_list = []

for e in range(num_epochs):
  train_acc = 0.0
  test_acc = 0.0
    
  model.train()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
    optimizer.zero_grad()
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)

    loss = loss_fn(out, label)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    optimizer.step()
    scheduler.step()
    train_acc += calc_accuracy(out, label)
    if batch_id % log_interval == 0:
      print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
      train_list.append(train_acc / (batch_id+1))
      loss_list.append(loss.data.cpu().numpy())
    
  print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
  
  model.eval() # 평가 모드로 변경
  
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)

  print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
  test_list.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


  0%|          | 0/1219 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.181069850921631 train acc 0.15625
epoch 1 batch id 201 loss 1.7890901565551758 train acc 0.1837686567164179
epoch 1 batch id 401 loss 1.6858924627304077 train acc 0.29800498753117205
epoch 1 batch id 601 loss 1.09676992893219 train acc 0.3694883527454243
epoch 1 batch id 801 loss 1.27403724193573 train acc 0.413623595505618
epoch 1 batch id 1001 loss 1.2889355421066284 train acc 0.4416208791208791
epoch 1 batch id 1201 loss 1.5640711784362793 train acc 0.4621149042464613
epoch 1 train acc 0.46305885972108285


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 2 test acc 0.6013934426229508


  0%|          | 0/1219 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.9162946343421936 train acc 0.71875
epoch 3 batch id 201 loss 1.0461186170578003 train acc 0.6441231343283582
epoch 3 batch id 401 loss 1.0902125835418701 train acc 0.6557824189526185
epoch 3 batch id 601 loss 0.7756926417350769 train acc 0.6689891846921797
epoch 3 batch id 801 loss 0.7322273254394531 train acc 0.6767712234706617
epoch 3 batch id 1001 loss 0.8623371124267578 train acc 0.6822864635364635
epoch 3 batch id 1201 loss 1.0786670446395874 train acc 0.6863551207327228
epoch 3 train acc 0.6863463904840033


  0%|          | 0/305 [00:00<?, ?it/s]

epoch 3 test acc 0.6152049180327869


In [20]:
# generate a scripted model

scripted = torch.jit.script(model, [token_ids, valid_length, segment_ids])

/usr/local/lib/python3.7/dist-packages/torch/jit/_script.py:931: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  "`optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead"


RuntimeError: ignored

In [16]:
# test
import torch

# load model
model1 = torch.load('kobert_v2.pt', map_location=torch.device('cpu'))
model1.load_state_dict(torch.load('kobert_v2_state_dict.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [17]:
import numpy as np

# flipped emotion dict
emotion_labels = {'중립': 0,
                  '기쁨': 1,
                  '놀람': 2,
                  '긴장됨': 3,
                  '괴로움': 4,
                  '화남': 5,
                  '비참함': 6,
                  '우울함': 7,
                  '피로함': 8}
flip_emotion_labels = {v: k for k, v in emotion_labels.items()}

def predict(predict_sentence):

    max_len = 64
    batch_size = 32

    data =[predict_sentence, '0']
    dataset_new = [data]

    new_test = EmotionDataset(dataset_new, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(new_test, batch_size=batch_size, num_workers=4)

    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long()
        segment_ids = segment_ids.long()

        valid_length = valid_length
        label = label.long()

        out1 = model1(token_ids, valid_length, segment_ids)

        for i in out1:
            logits = i
            logits = logits.detach().cpu().numpy()

            logits_zero = [np.round(x, 3) if x > 0 else 0 for x in logits]
            print('kobert_v2 :', logits_zero, flip_emotion_labels[np.argmax(logits)])

            return flip_emotion_labels[np.argmax(logits)]

In [20]:
max_len = 64
batch_size = 32

data =['안녕하세요 테스트 문장입니다', '0']
dataset_new = [data]

new_test = EmotionDataset(dataset_new, 0, 1, tok, vocab, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(new_test, batch_size=batch_size, num_workers=4)

for token_ids, valid_length, segment_ids, label in test_dataloader:   
    x = token_ids, valid_length, segment_ids
    y = model1(token_ids, valid_length, segment_ids)

    print(type(token_ids))

    out1 = model1(token_ids, valid_length, segment_ids)

    print(out1)

<class 'torch.Tensor'>
tensor([[ 3.8171, -2.0730, -0.8213, -0.1868, -0.6384, -0.4231,  3.3142, -0.6208,
         -0.9593]], grad_fn=<AddmmBackward>)


In [ ]:
# optimize the model for mobile inference
from torch.utils.mobile_optimizer import optimize_for_mobile

optimized_model = optimize_for_mobile(scripted)

In [18]:
predict('안녕하세요 테스트 문장입니다')

kobert_v2 : [3.817, 0, 0, 0, 0, 0, 3.314, 0, 0] 중립


'중립'